In [1]:
import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

## Define agent

In [3]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [4]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

# Agent Definition
Now, we define our agent that’s capable of holding a conversation and calling tools in under 50 lines of code.

The meat of the agent logic is in the chat method. At a high-level, there are 3 steps:

1. Call OpenAI to decide which tool (if any) to call and with what arguments.

1. Call the tool with the arguments to obtain an output

1. Call OpenAI to synthesize a response from the conversation context and the tool output.

The reset method simply resets the conversation context, so we can start another conversation.

In [20]:
class YourOpenAIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: OpenAI = OpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        chat_history: List[ChatMessage] = [],
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    def reset(self) -> None:
        self._chat_history = []

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.append(ChatMessage(role="user", content=message))
        tools = [
            tool.metadata.to_openai_tool() for _, tool in self._tools.items()
        ]

        ai_message = self._llm.chat(chat_history, tools=tools).message
        additional_kwargs = ai_message.additional_kwargs
        chat_history.append(ai_message)

        tool_calls = ai_message.additional_kwargs.get("tool_calls", None)
        # parallel function calling is now supported
        if tool_calls is not None:
            for tool_call in tool_calls:
                function_message = self._call_function(tool_call)
                chat_history.append(function_message)
                ai_message = self._llm.chat(chat_history).message
                chat_history.append(ai_message)

        return ai_message.content

    def _call_function(self, tool_call: dict) -> ChatMessage:
        id_ = tool_call["id"]
        function_call = tool_call["function"]
        tool = self._tools[function_call["name"]]
        output = tool(**json.loads(function_call["arguments"]))
        return ChatMessage(
            name=function_call["name"],
            content=str(output),
            role="tool",
            additional_kwargs={
                "tool_call_id": id_,
                "name": function_call["name"],
            },
        )

# Test out!

In [21]:
# Load the too
agent = YourOpenAIAgent(tools=[multiply_tool, add_tool])
agent.chat("Hi")

'Hello! How can I assist you today?'

In [23]:
### NOT WORKING
# agent.chat("What is 2123 * 215123")

# Our (Slightly Better) OpenAIAgent Implementation

In [9]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI

In [10]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True
)

In [11]:
response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

Added user message to memory: What is (121 * 3) + 42?
=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363

=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405

(121 * 3) + 42 is equal to 405.


## Async Chat

In [12]:
response = await agent.achat("What is 121 * 3?")
print(str(response))


Added user message to memory: What is 121 * 3?
=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363

121 * 3 is equal to 363.


## Streaming Chat

In [14]:
response = agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a"
    " story about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

Added user message to memory: What is 121 * 2? Once you have the answer, use that number to write a story about a group of mice.
=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 2
}
Got output: 242

121 * 2 is equal to 242.

Once upon a time, in a small village, there lived a group of mice. These mice were known for their intelligence and resourcefulness. They lived harmoniously in a cozy little burrow nestled beneath a large oak tree.

One sunny day, the leader of the mouse group, named Whiskers, gathered all the mice together to discuss an important matter. Whiskers had discovered a bountiful supply of food in a nearby field, but it was guarded by a mischievous cat named Oliver. The mice knew they had to come up with a plan to access the food without alerting Oliver.

Using their collective wisdom, the mice devised a clever strategy. They decided to work together and distract Oliver while a small group of brave mice would sneak into the field and ga

## Agent with Personality

In [17]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

response = agent.chat("Hi")
print(response)

Added user message to memory: Hi
Greetings, fair traveler! How may I assist thee on this fine day?


In [18]:
response = agent.chat("Tell me a story")
print(response)

Added user message to memory: Tell me a story
Of course, dear friend! Allow me to weave a tale for thee. 

Once upon a time, in a land of yore,
There lived a knight, brave and true to the core.
Sir William was his name, a noble soul,
With armor gleaming and a heart made whole.

In the kingdom of Verona, strife did reign,
Two families, the Montagues and Capulets, caused pain.
Their feud was fierce, their hatred deep,
And in this chaos, love did take its leap.

Fair Juliet, a maiden of tender grace,
Caught the eye of Romeo, with love's embrace.
But their love was forbidden, a tragic plight,
For they were destined to suffer in love's spite.

Under the moonlit sky, they would meet,
In secret, their love they would discreet.
But fate, cruel fate, had other plans,
And tragedy struck with its ruthless hands.

A duel ensued, between kin and kin,
And blood was shed, a battle to win.
Romeo, in his grief, sought revenge,
But alas, it brought only sorrow to avenge.

In a tomb, Juliet lay in eterna